In [14]:
import sys
from mcap_protobuf.decoder import DecoderFactory
from mcap.reader import make_reader
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def main():
    files = [
        "/home/kuribayashi-a/Downloads/follow_ego_80_other_60.mcap", "yatagarasu",
        "/home/kuribayashi-a/Downloads/follow_ego_80_other_60(1).mcap", "apollo",
    ]
    start_time = 1749644075851999042
    start_time /= 1e9
    end_time = 1749644077475000000
    end_time /= 1e9

    is_ad_mode = False
    is_time = False
    is_plan_data_obtained = False
    time = 0.0
    prev_time = 0.0
    plan_data = pd.DataFrame([], columns=[
        "time", 
        "v", 
        "a",
        "lateral_error", 
        "kappa",
        "x", 
        "y",
        "label"
        ])
    chassis_data = pd.DataFrame([], columns=[
        "time", 
        "v", 
        "label"
        ])

    control_data = pd.DataFrame([], columns=[
        "time", 
        "a", 
        "lateral_error",
        "label"
        ])

    
    for i in range(0, len(files), 2):      
        with open(files[i], "rb") as f:
            reader = make_reader(f, decoder_factories=[DecoderFactory()])
            for schema, channel, message, proto_msg in reader.iter_decoded_messages():
        
                # print(is_time)
        
                if channel.topic == "/apollo/canbus/chassis":
                    is_ad_mode = (proto_msg.driving_mode == 1)
        
                if not (is_time and is_ad_mode):
                    continue
        
                if channel.topic == "/apollo/planning":
                    header_time = proto_msg.header.timestamp_sec
                    is_time = start_time < header_time < end_time
                    is_plan_data_obtained = True
                    df = pd.DataFrame([[
                        message.log_time,
                        proto_msg.only_trajectory[0].v,
                        proto_msg.only_trajectory[0].a,
                        proto_msg.lane_change_data.lat_data.s[0],
                        proto_msg.only_trajectory[0].path_point.kappa,
                        proto_msg.only_trajectory[0].path_point.x,
                        proto_msg.only_trajectory[0].path_point.y,
                        sys.argv[i-1],
                        ]], columns = plan_data.columns)
                    plan_data = pd.concat([plan_data, df])
        
                if not is_plan_data_obtained and not is_time:
                    continue
        
                if channel.topic == "/apollo/control":
                    df = pd.DataFrame([[
                        message.log_time,
                        proto_msg.acceleration,
                        proto_msg.debug.mpc_status.lateral_error,
                        sys.argv[i-1],
                        ]], columns = control_data.columns)
                    control_data = pd.concat([control_data, df])
        
                if channel.topic == "/apollo/chassis":
                    df = pd.DataFrame([[
                        message.log_time,
                        proto_msg.speed_mps,
                        sys.argv[i-1],
                        ]], columns = chassis_data.columns)
                    chassis_data = pd.concat([chassis_data, df])
        
                is_plan_data_obtained = False

    plot_data(plan_data, "time", "x", "x.svg")

def plot_data(data, x, y, title):
    fix, ax = plt.subplots()
    sns.lineplot(data=data, x=x, y=y, hue="label")
    fig.regend()
    plt.legend(loc="lower center", bbox_to_anchor=(-1, 1.0), ncol=4)
    plt.savefig(f"{title}.svg")
    
if __name__ == "__main__":
    main()


DecoderNotFoundError: no decoder factory supplied for message encoding cdr, schema Schema(id=1, data=b'================================================================================\nIDL: process_manager_interfaces/msg/ProcessGroupInfo\n#include "apex_msgs/msg/AppInfo.idl"\n#include "process_manager_interfaces/msg/ProcessInfo.idl"\n\nmodule process_manager_interfaces {\n  module msg {\n    module ProcessGroupInfo_Enums {\n        enum ErrorType {\n          OK,\n          ERROR,\n          FATAL\n      };\n    };\n\n    @verbatim (language="comment", text=\n      "Message with information about a process group")\n    struct ProcessGroupInfo {\n     @verbatim (language="comment", text=\n       "Name of the process group ")\n     string<255> process_group_name;\n\n     @verbatim (language="comment", text=\n       "Source information")\n     apex_msgs::msg::AppInfo src_info;\n\n     @verbatim (language="comment", text=\n       "Name of the current process group state")\n     string<255>